<a href="https://colab.research.google.com/github/NirjharDebnath/Machine-Learning/blob/main/MNISTFASHION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from timeit import default_timer as timer
from helper_fn import accuracy_fn
from tqdm.auto import tqdm

print(torch.__version__)
print(torchvision.__version__)

2.5.0+cu124
0.20.0+cu124


/home/nirjhar/Python Codes/Machine Learning/torchenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
device = 'cuda' if torch.cuda.is_available else 'cpu'
device

'cuda'

In [ ]:
train_data = torchvision.datasets.FashionMNIST(root="Data", download=True, train=True, transform=transforms.ToTensor(),target_transform=None)
test_data = torchvision.datasets.FashionMNIST(root="Data", download=True, train=False, transform=transforms.ToTensor(),target_transform=None)


In [ ]:
class_to_idx = train_data.class_to_idx
class_to_idx
class_names = train_data.classes
class_names

['T-shirt/top',
 'Trouser',
 'Pullover',
 'Dress',
 'Coat',
 'Sandal',
 'Shirt',
 'Sneaker',
 'Bag',
 'Ankle boot']

In [ ]:
from torch.utils.data import DataLoader

BATCH_SIZE = 32
train_dataloader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(dataset=test_data, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
train_features_batch, train_labels_batch = next(iter(train_dataloader))
train_features_batch.shape, train_labels_batch.shape

(torch.Size([32, 1, 28, 28]), torch.Size([32]))

## Model

In [ ]:
class ModelCNN(nn.Module):
    def __init__(self, input_shape, output_shape, hidden_units):
        super().__init__()
        self.convblock1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape,
                      out_channels=hidden_units,
                      kernel_size=3,
                      padding=1,
                      stride=1),
            nn.ReLU(),

            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      padding=1,
                      stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.convblock2 = nn.Sequential(
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      padding=1,
                      stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden_units*7*7,
                      out_features=output_shape)
        )

    def forward(self, x):

        x = self.convblock1(x)
        # print(x.shape)
        x = self.convblock2(x)
        # print(x.shape)
        x = self.classifier(x)
        return(x)

In [ ]:
torch.manual_seed(42)

model = ModelCNN(input_shape=1,
                 output_shape=len(class_names),
                 hidden_units=10).to(device)

In [ ]:
lossfn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model.parameters(), lr=0.1)

In [ ]:
image = torch.randn(size=(1,28,28)).to(device)
out = model(image.unsqueeze(dim=0))
out.shape

torch.Size([1, 10])

In [ ]:
def train_step(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               accuracy_fn,
               device: torch.device = device):
    train_loss, train_acc = 0, 0
    model.to(device)

    for batch, (X, y) in enumerate(data_loader):
        # Send data to GPU
        X, y = X.to(device), y.to(device)

        # 1. Forward pass
        y_pred = model(X)

        # 2. Calculate loss
        loss = loss_fn(y_pred, y)
        train_loss += loss
        train_acc += accuracy_fn(y_true=y,
                                 y_pred=y_pred.argmax(dim=1)) # Go from logits -> pred labels

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

    # Calculate loss and accuracy per epoch and print out what's happening
    train_loss /= len(data_loader)
    train_acc /= len(data_loader)
    print(f"Train loss: {train_loss:.5f} | Train accuracy: {train_acc:.2f}%")

def test_step(data_loader: torch.utils.data.DataLoader,
              model: torch.nn.Module,
              loss_fn: torch.nn.Module,
              accuracy_fn,
              device: torch.device = device):
    test_loss, test_acc = 0, 0
    model.to(device)
    model.eval() # put model in eval mode
    # Turn on inference context manager
    with torch.inference_mode():
        for X, y in data_loader:
            # Send data to GPU
            X, y = X.to(device), y.to(device)

            # 1. Forward pass
            test_pred = model(X)

            # 2. Calculate loss and accuracy
            test_loss += loss_fn(test_pred, y)
            test_acc += accuracy_fn(y_true=y,
                y_pred=test_pred.argmax(dim=1) # Go from logits -> pred labels
            )

        # Adjust metrics and print out
        test_loss /= len(data_loader)
        test_acc /= len(data_loader)
        print(f"Test loss: {test_loss:.5f} | Test accuracy: {test_acc:.2f}%\n")

In [ ]:
torch.manual_seed(42)

# Measure time
from timeit import default_timer as timer
timeron = timer()

# Train and test model
EPOCHS = 4
for epoch in tqdm(range(EPOCHS)):
    print(f"Epoch: {epoch}\n---------")
    train_step(data_loader=train_dataloader,
        model=model,
        loss_fn=lossfn,
        optimizer=optimizer,
        accuracy_fn=accuracy_fn,
        device=device
    )
    test_step(data_loader=test_dataloader,
        model=model,
        loss_fn=lossfn,
        accuracy_fn=accuracy_fn,
        device=device
    )

timeroff = timer()
print(f"Total time taken on {device} : {timeroff-timeron}")


  0%|          | 0/4 [00:00<?, ?it/s]

Epoch: 0
---------
Train loss: 0.63393 | Train accuracy: 77.39%


 25%|██▌       | 1/4 [00:07<00:21,  7.16s/it]

Test loss: 0.40455 | Test accuracy: 85.51%

Epoch: 1
---------
Train loss: 0.38117 | Train accuracy: 86.17%


 50%|█████     | 2/4 [00:14<00:14,  7.03s/it]

Test loss: 0.37164 | Test accuracy: 86.55%

Epoch: 2
---------
Train loss: 0.34345 | Train accuracy: 87.50%


 75%|███████▌  | 3/4 [00:20<00:06,  6.97s/it]

Test loss: 0.33835 | Test accuracy: 88.08%

Epoch: 3
---------
Train loss: 0.31915 | Train accuracy: 88.47%


100%|██████████| 4/4 [00:28<00:00,  7.01s/it]

Test loss: 0.33208 | Test accuracy: 88.16%

Total time taken on cuda : 28.040696779999053


In [ ]:
from helpers import eval_model
# print(device)
modelresults = eval_model(model=model,
                          data_loader=test_dataloader,
                          loss_fn=lossfn,
                          accuracy_fn=accuracy_fn,
                          device=device)
print(modelresults)

{'model_name': 'ModelCNN', 'model_loss': 0.33208173513412476, 'model_acc': 88.15894568690096}


In [ ]:
model._save_to_state_dict

<bound method Module._save_to_state_dict of ModelCNN(
  (convblock1): Sequential(
    (0): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (convblock2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=490, out_features=10, bias=True)
  )
)>

In [ ]:
import torch
import torchvision.transforms as transforms
from PIL import Image

def image_to_tensor(image_path):
    """
    Converts a saved image to a PyTorch tensor.

    Args:
        image_path (str): Path to the image file.

    Returns:
        torch.Tensor: The image as a PyTorch tensor.
                      Returns None if there's an error loading or processing the image.
    """
    try:
        # 1. Open the image using PIL
        image = Image.open(image_path)

        # 2. Define transformations (optional, but highly recommended)
        #    - Convert to RGB (if the image is grayscale or has an alpha channel)
        #    - Resize to a consistent size (if needed for your model)
        #    - Convert to tensor
        #    - Normalize pixel values (usually between 0 and 1 or -1 and 1, depending on the model)
        transform = transforms.Compose([
            transforms.Grayscale(),
            transforms.ToTensor(),  # Convert to tensor
            transforms.Resize((28, 28)), # Resize to a common size (adjust as needed) If removed, keep transforms.CenterCrop below
            # transforms.CenterCrop(28), # Center crop to the specified size
            # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize using ImageNet stats (adjust if using a different dataset)
        ])



        # 3. Apply the transformations
        tensor = transform(image)


        return tensor

    except (FileNotFoundError, OSError, ValueError) as e:  # Catch potential errors like file not found, or image format issues
        print(f"Error loading or processing image: {e}")
        return None



# Example usage:
image_path = "/home/nirjhar/Python Codes/Machine Learning/shoe2.png" # Replace with the actual path to your image
image_tensor = image_to_tensor(image_path)

if image_tensor is not None:
    print(f"Image tensor shape: {image_tensor.shape}")
    print(f"Image tensor data type: {image_tensor.dtype}")


Image tensor shape: torch.Size([1, 28, 28])
Image tensor data type: torch.float32


In [ ]:
image_tensor = image_tensor.to(device)

In [ ]:
output = model(image_tensor.unsqueeze(dim=0))
output.argmax(dim=1)


tensor([9], device='cuda:0')

In [ ]:
class_to_idx

{'T-shirt/top': 0,
 'Trouser': 1,
 'Pullover': 2,
 'Dress': 3,
 'Coat': 4,
 'Sandal': 5,
 'Shirt': 6,
 'Sneaker': 7,
 'Bag': 8,
 'Ankle boot': 9}